In [1]:
!pip install selenium
!apt-get update #to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
import csv, ssl, time, urllib, urllib.request

     |████████████████████████████████| 911kB 2.8MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [854 kB]
Get:12 http://s

In [0]:
wd_for_attractions = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
wd_for_single = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
site_file = open('Hamburg_fixed[3].csv', 'w')
site_writer = csv.writer(site_file)
site_writer.writerow(['Name', 'Ranking', 'A_Type','B_Type', 'Date', 'Country'])
unknown_spelling_counter= 0
#tx = ssl.create_default_context()
#ctx.check_hostname = False
#ctx.verify_mode = ssl.CERT_NONE

wd_for_attractions.get('https://www.tripadvisor.de/Attractions-g187331-Activities-oa330-Hamburg.html')

while True:
    
    continue_attractions = 0
    time.sleep(1)
    attractions_soup = BeautifulSoup(wd_for_attractions.page_source, 'html.parser')
    attractions_in_list = attractions_soup.find_all(attrs={'class':'_6sUF3jUd'}) 

    for attraction_in_list in attractions_in_list:

        try:
            Name = attraction_in_list.find(attrs={'class':'_1QKQOve4'}).text
        except:
            Name = None
        
        try:
            attraction_in_list.find(attrs={'class':'_82HNRypW'}).text
        except:
            print('No Reviews for '+Name)
            continue
        
        try:
            A_Type = attraction_in_list.find(attrs={'class':'_21qUqkJx'}).text
        except:
            A_Type = None

        attraction_link = 'https://www.tripadvisor.de' + attraction_in_list.find(attrs={'class':'_1QKQOve4'}).get('href')
        wd_for_single.get(attraction_link)
        single_soup = BeautifulSoup(wd_for_single.page_source, 'html.parser')
    
        try:
            Ranking = single_soup.find(attrs={'class':'attractions-attraction-review-header-attraction-review-header__popIndex--H1_nL'}).text.split()[1]
        except:
            Ranking = None

    
        try:
            B_Type = single_soup.find(attrs={'class':'attractions-attraction-review-header-AttractionLinks__dotted_link--Pt2MP'}).text
        except:
            B_Type = None
    
        print(Name, A_Type, B_Type, Ranking)
    

        for i in range(20):
            try:
                wd_for_single.find_elements_by_css_selector("li.ui_radio:nth-child(1) > label:nth-child(2) > span:nth-child(1)").click()
            except:
              print('retry all languages')
              time.sleep(1)
        break_counter = 0          
        while True:
            time.sleep(1)
            continuer = False
            soup = BeautifulSoup(wd_for_single.page_source, 'html.parser')
            reviews = soup.find_all(attrs={'class':'location-review-card-Card__ui_card--2Mri0 location-review-card-Card__card--o3LVm location-review-card-Card__section--NiAcw'})
            for review in reviews:

                try:
                    Country = review.find(attrs={'class':'default social-member-common-MemberHometown__hometown--3kM9S small'}).text.split(',')[1:]
                except:
                    Country = None

                try:  
                    link_to_prof = review.find(attrs = {'class':'location-review-review-list-parts-ReviewTitle__reviewTitleText--2tFRT'}).get('href')
                    url = 'https://www.tripadvisor.de'+link_to_prof
                    file = urllib.request.urlopen(url)
                    review_dirty = BeautifulSoup(file, 'html.parser')
                    review_clean = review_dirty.find(attrs={'class':'featured-review-container'})
                    Date = review_clean.find(attrs={'class':'ratingDate'}).text.split()[1:4]
                except:
                    Date=None

                if Date is not None:
                  try:
                    if int(Date[2]) <= 2018:
                      break_counter = break_counter + 1
                  except: pass

                try:
                    site_writer.writerow([Name, Ranking, A_Type, B_Type, Date, Country])
                except:
                    unknown_spelling_counter = unknown_spelling_counter + 1
                    print(Name, Ranking, A_Type, B_Type, Date, Country)
            
            if break_counter >= 5:
                break
            
            for i in range(3):
                try:
                    wd_for_single.find_element_by_css_selector("a.ui_button:nth-child(2)").click()
                    continuer = True
                    break
                except:
                    time.sleep(1)

                    
            if continuer==False:
                break
            
        

    for i in range(3):
        try:
            wd_for_attractions.find_element_by_css_selector("a.ui_button").click()
            continue_attractions = continue_attractions + 1
            print('continues to next page')
            break
        except:
            print("retry next page")
            time.sleep(1)
    
    if continue_attractions==12:
        break

site_file.close()
wd_for_attractions.close()
wd_for_single.close()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: use options instead of chrome_options
  


Streaming output truncated to the last 5000 lines.
retry all languages
retry all languages
retry all languages
retry all languages
retry all languages
retry all languages
retry all languages
retry all languages
retry all languages
retry all languages
retry all languages
retry all languages
retry all languages
retry all languages
Colonnaden Sehenswürdigkeiten & Wahrzeichen Wahrzeichen & Sehenswürdigkeiten 78
retry all languages
retry all languages
retry all languages
retry all languages
retry all languages
retry all languages
retry all languages
retry all languages
retry all languages
retry all languages
retry all languages
retry all languages
retry all languages
retry all languages
retry all languages
retry all languages
retry all languages
retry all languages
retry all languages
retry all languages
Jenischpark Natur & Parks Parks 75
retry all languages
retry all languages
retry all languages
retry all languages
retry all languages
retry all languages
retry all languages
retry all lang

In [0]:
from google.colab import drive
drive.mount('/content/drive')